In [1]:
# To disable `warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)`
import warnings
warnings.filterwarnings('ignore')

## 5. Disciplined Geometric Programming

__Disciplined geometric programming (DGP)__ is an analog of DCP for log-log convex functions, that is, functions of positive variables that are convex with respect to the geometric mean instead of the arithmetic mean.

While DCP is a ruleset for constructing convex programs, DGP is a ruleset for __log-log convex programs (LLCPs)__, which are problems that are convex after the variables, objective functions, and constraint functions are replaced with their logs, an operation that we refer to as a log-log transformation. <u>Every __geometric program (GP)__ and __generalized geometric program (GGP)__ is an LLCP</u>, but there are LLCPs that are neither GPs nor GGPs.

CVXPY lets you form and solve DGP problems, just as it does for DCP problems. For example, the following code solves a simple geometric program,

In [2]:
import cvxpy as cp

# DGP requires Variables to be declared positive via `pos=True`.
x = cp.Variable(pos=True)
y = cp.Variable(pos=True)
z = cp.Variable(pos=True)

objective = x * y * z
constraints = [
    4 * x * y * z + 2 * x * z <= 10,
    x <= 2 * y,
    y <= 2 * x,
    z >= 1
]
prob = cp.Problem(cp.Maximize(objective), constraints)
prob.solve(gp=True)

print(f"""
optimal value: {prob.value}
x: {x.value}
y: {y.value}
z: {z.value}
""")


optimal value: 1.9999999938309496
x: 0.9999999989682057
y: 1.999999974180587
z: 1.0000000108569758



Note that to solve DGP problems, you must pass the option `gp=True` to the `solve()` method.

This section explains what DGP is, and it shows how to construct and solve DGP problems using CVXPY. At the end of the section are tables listing all the atoms that can be used in DGP problems, similar to the tables presented in the section on [DCP atoms](https://www.cvxpy.org/tutorial/functions/index.html#functions).

### Log-log curvature

Just as every Expression in CVXPY has a curvature (constant, affine, convex, concave, or unknown), every Expression also has a __log-log curvature__.

A function $f\colon D\subseteq\mathbb{R}^n_{++}\to\mathbb{R}$ is said to be __log-log convex__ if the function $F(u)=\log f(e^u)$, with domain $\{u\in\mathbb{R}^n\mid e^u\in D\}$, is convex (where $\mathbb{R}^n_{++}$ denotes the set of positive reals and the logarithm and exponential are meant elementwise).

The function $F$ is called the __log-log transformation__ of $f$. The function $f$ is __log-log concave__ if $F$ is concave, and it is __log-log affine__ if $F$ is affine.

Every log-log affine function (called a __monomial function__ in GP) has the form
$$
f(x) = cx_1^{a_1}x_2^{a_2}\dotsb x_n^{a_n}
$$
where $x$ is in $\mathbb{R}^n_{++}$, the $a_i$ are real numbers, and $c$ is a positive scalar.

A sum of monomials, known as a __posynomial function__ in GP, is a log-log convex function; A table of all the [atoms with known log-log curvature](https://www.cvxpy.org/tutorial/dgp/index.html#dgp-atoms) is presented at the end of this page.

In the below table, $F$ is the log-log transformation of $f$, $u=\log x$, and $v=\log y$, where $x$ and $y$ are in the domain of $f$.

![](figures/log-log_curvature.png)

CVXPY's log-log curvature analysis can flag Expressions as unknown even when they are log-log convex or log-log concave. Note that any log-log constant expression is also log-log affine, and any log-log affine expression is log-log convex and log-log concave.

The log-log curvature of an Expression is stored in its `.log_log_curvature` attribute. For example, running the following script

In [3]:
x = cp.Variable(pos=True)
y = cp.Variable(pos=True)

constant = cp.Constant(2.0)
monomial = constant * x * y
posynomial = monomial + (x ** 1.5) * (y ** -1)
reciprocal = posynomial ** -1
unknown = reciprocal + posynomial

constant.log_log_curvature, monomial.log_log_curvature, \
posynomial.log_log_curvature, reciprocal.log_log_curvature, \
unknown.log_log_curvature

('LOG-LOG CONSTANT',
 'LOG-LOG AFFINE',
 'LOG-LOG CONVEX',
 'LOG-LOG CONCAVE',
 'UNKNOWN')

You can also check the log-log curvature of an Expression by calling the methods `is_log_log_constant()`, `is_log_log_affine()`, `is_log_log_convex()`, `is_log_log_concave()`. For example

In [4]:
posynomial.is_log_log_convex()

True

### Log-log curvature rules

For an Expression to have known log-log curvature, all of the Constants, Variables, and Parameters it refers to must be elementwise positive. A Constant is positive if its numerical value is positive. Variables and Parameters are positive only if the keyword argument `pos=True` is supplied to their constructors (e.g., `x = cp.Variable(shape=(), pos=True)`). To summarize, when formulating a DGP problem, all Constants should be elementwise positive, and all Variables and Parameters must be constructed with the attribute `pos=True`.

DGP analysis is exactly analogous to DCP analysis. It is based on a library of atoms (functions) with known monotonicity and log-log curvature and a a single composition rule. The [library of atoms](https://www.cvxpy.org/tutorial/dgp/dgp-atoms) is presented at the end of this page; the composition rule is stated below.

A function $f(\text{expr}_1,\text{expr}_2,\dotsc,\text{expr}_n)$ is log-log convex if $f$ is a log-log convex function and for each $\text{expr}_i$ one of the following conditions holds:

- $f$ is increasing in argument $i$ and $\text{expr}_i$ is log-log convex.
- $f$ is decreasing in argument $i$ and $\text{expr}_i$ is log-log concave.
- $\text{expr}_i$ is log-log affine.

A function $f(\text{expr}_1,\text{expr}_2,\dotsc,\text{expr}_n)$ is log-log concave if $f$ is a log-log concave function and for each $\text{expr}_i$ one of the following conditions holds:

- $f$ is increasing in argument $i$ and $\text{expr}_i$ is log-log concave.
- $f$ is decreasing in argument $i$ and $\text{expr}_i$ is log-log convex.
- $\text{expr}_i$ is log-log affine.

A function $f(\text{expr}_1,\text{expr}_2,\dotsc,\text{expr}_n)$ is log-log affine if $f$ is an log-log affine function and each $\text{expr}_i$ is log-log affine.

If none of the three rules apply, the expression $f(\text{expr}_1,\text{expr}_2,\dotsc,\text{expr}_n)$ is marked as having unknown curvature.

If an Expression satisfies the composition rule, we colloquially say that the Expression "is DGP." You can check whether an Expression is DGP by calling the method `is_dgp()`. For example, the assertions in the following code block will pass.

In [5]:
x = cp.Variable(pos=True)
y = cp.Variable(pos=True)

monomial = 2.0 * constant * x * y
posynomial = monomial + (x ** 1.5) * (y ** -1)

assert monomial.is_dgp()
assert posynomial.is_dgp()

An Expression is DGP precisely when it has known log-log curvature, which means at least one of the methods `is_log_log_constant()`, `is_log_log_affine()`, `is_log_log_convex()`, `is_log_log_concave()` will return `True`.

### DGP problems

A Problem is constructed from an objective and a list of constraints. If a problem follows the DGP rules, it is guaranteed to be an LLCP and solvable by CVXPY. The DGP rules require that the problem objective have one of two forms:

- Minimize(log-log convex)
- Maximize(log-log concave)

The only valid constraints under the DGP rules are

- log-log affine `==` log-log affine
- log-log convex `<=` log-log concave
- log-log concave `>=` log-log convex

You can check that a problem, constraint, or objective satisfies the DGP rules by calling `object.is_dgp()`. Here are some examples of DGP and non-DGP problems:

In [6]:
# DGP requires Variables to be declared positive via `pos=True`.
x = cp.Variable(pos=True)
y = cp.Variable(pos=True)
z = cp.Variable(pos=True)

objective = x * y * z
constraints = [
    4 * x * y * z + 2 * x * z <= 10,
    x <= 2 * y,
    y <= 2 * x,
    z >= 1
]
assert objective.is_log_log_concave()
assert all(constraint.is_dgp() for constraint in constraints)

prob = cp.Problem(cp.Maximize(objective), constraints)
assert prob.is_dgp()

# All Variables must be declared as positive for an Expression to be DGP.
w = cp.Variable()
objective = w * x * y
assert not objective.is_dgp()

prob = cp.Problem(cp.Maximize(objective), constraints)
assert not prob.is_dgp()

CVXPY will raise an exception if you call `prob.solve(gp=True)` on a non-DGP problem.

### DGP atoms

This section of the tutorial describes the DGP atom library, that is, the atomic functions with known log-log curvature and monotonicity. CVXPY uses the function information in this section and the DGP rules to mark expressions with a log-log curvature. Note that every DGP expression is positive.

#### Infix operators
The infix operators `+`, `*`, `/` are treated as atoms. The operators `*` and `/` are log-log affine functions. The operator `+` is log-log convex in both its arguments.

Note that in CVXPY, `expr1 * expr2` denotes matrix multiplication when `expr1` and `expr2` are matrices; if you're running Python 3, you can alternatively use the `@` operator for matrix multiplication. Regardless of your Python version, you can also use the `cp.matmul` atom to multiply two matrices. To multiply two arrays or matrices elementwise, use the `cp.multiply` atom. Finally, to take the product of the entries of an Expression, use the `cp.prod` atom.

#### Transpose
The transpose of any expression can be obtained using the syntax `expr.T`. Transpose is a log-log affine function.

#### Power
For any CVXPY expression `expr`, the power operator `expr**p` is equivalent to the function `cp.power(expr, p)`. Taking powers is a log-log affine function.

#### Scalar functions
A scalar function takes one or more scalars, vectors, or matrices as arguments and returns a scalar. Note that several of these atoms may be applied along an axis; see the API reference or the [DCP atoms tutorial](https://www.cvxpy.org/tutorial/functions/index.html#functions) for more information.

![](figures/dgp_scalar_functions.png)

#### Elementwise functions
These functions operate on each element of their arguments. For example, if `X` is a 5 by 4 matrix variable, then `cp.sqrt(X)` is a 5 by 4 matrix expression. `cp.sqrt(X)[1, 2]` is equivalent to `cp.sqrt(X[1, 2])`.

Elementwise functions that take multiple arguments, such as `cp.maximum` and `cp.multiply`, operate on the corresponding elements of each argument. For example, if `X` and `Y` are both 3 by 3 matrix variables, then `cp.maximum(X, Y)` is a 3 by 3 matrix expression. `cp.maximum(X, Y)[2, 0]` is equivalent to `cp.maximum(X[2, 0], Y[2, 0])`. This means all arguments must have the same dimensions or be scalars, which are promoted.

![](figures/dgp_elementwise_functions.png)

#### Vector/matrix functions
A vector/matrix function takes one or more scalars, vectors, or matrices as arguments and returns a vector or matrix.

![](figures/dgp_vector_matrix_functions.png)